<a href="https://colab.research.google.com/github/lg550055/102-lab05/blob/main/yhooFin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import re
import json
import requests
headers={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) (KHTML, like Gecko) Chrome/102.0.5005.63'}
ticker = 'META'

In [2]:
url = 'https://finance.yahoo.com/quote/'+ ticker + '/analysis?p=' + ticker
page = requests.get(url, headers=headers, timeout=5)
data = json.loads(re.search('root\.App\.main\s*=\s*(.*);', page.text).group(1))["context"]["dispatcher"]["stores"]
# only the data["QuoteSummaryStore"]['earningsTrend']['trend'] object has eps and rev estimates

In [ ]:
data['QuoteSummaryStore'].keys()

In [ ]:
data['QuoteSummaryStore']['financialData'] # good snapshot summary
data['QuoteSummaryStore']['summaryDetail'] # div, and x-div date, highs and lows
data['QuoteSummaryStore']['industryTrend'] # empty
data['QuoteSummaryStore']['sectorTrend'] # empty

In [3]:
f = [t for t in data["QuoteSummaryStore"]["earningsTrend"]["trend"] if t["period"] in ['0y','+1y']]
num_analysts = f[0]['earningsEstimate']['numberOfAnalysts']['fmt']
fwd_eps = f[0]['earningsEstimate']['avg']['fmt']
fwd_rev = f[0]['revenueEstimate']['avg']['raw']
fwd_rev_g = f[0]['revenueEstimate']['growth']['raw']
fwd2_eps = f[1]['earningsEstimate']['avg']['fmt']
fwd2_rev = f[1]['revenueEstimate']['avg']['fmt']
fwd2_rev_g = f[1]['revenueEstimate']['growth']['fmt']
print(num_analysts, fwd_eps, fwd_rev, fwd_rev_g, fwd2_eps, fwd2_rev, fwd2_rev_g)
# print("Next 5 Years (per annum) : " + field["growth"]["fmt"])

11 13.37 8498660000 -0.12100001 15.16 8.49B -0.10%


In [16]:
# url for is, bs, cf -but not all items, e.g. q dil shrs or net debt
url = 'https://finance.yahoo.com/quote/'+ ticker + '/financials?p=' + ticker
page = requests.get(url, headers=headers, timeout=5)
data = json.loads(re.search('root\.App\.main\s*=\s*(.*);', page.text).group(1))["context"]["dispatcher"]["stores"]

In [ ]:
# data["RecommendationStore"]['recommendedSimilarSymbols'] # a few others in the sector
# data["QuoteSummaryStore"]['incomeStatementHistoryQuarterly']['incomeStatementHistory'] # IS items x shr ct
# data["QuoteSummaryStore"]['balanceSheetHistoryQuarterly']['balanceSheetStatements'] # BS items (but not explicit net debt)
# data["QuoteSummaryStore"]['earnings'] # Q and A rev and net income
# data["QuoteSummaryStore"]['summaryDetail'] # price, vol, div, p/e, p/s
# rev = data["QuoteSummaryStore"]['incomeStatementHistory']['incomeStatementHistory'][0]['totalRevenue']['fmt']
# eop = data["QuoteSummaryStore"]['incomeStatementHistory']['incomeStatementHistory'][0]['endDate']['fmt']
# bseop = data["QuoteSummaryStore"]['balanceSheetHistory']['balanceSheetStatements'][0]['endDate']['fmt']
# ebit = data["QuoteTimeSeriesStore"]['timeSeries']['annualEBIT'][-1]['reportedValue']['raw'] / 1e9
# dshrs = data["QuoteTimeSeriesStore"]['timeSeries']['annualDilutedAverageShares'][-1]['reportedValue']['raw'] / 1e9
# eps = data["QuoteTimeSeriesStore"]['timeSeries']['annualDilutedEPS'][-1]['reportedValue']['fmt']
data["QuoteTimeSeriesStore"]['timeSeries'].keys() # annual and trailing IS items

In [18]:
# IS, CF items for prev 4 fy and IS for TTM
date1 = data["QuoteTimeSeriesStore"]['timeSeries']['annualTotalRevenue'][-1]['asOfDate']
rev1 = data["QuoteTimeSeriesStore"]['timeSeries']['annualTotalRevenue'][-1]['reportedValue']['raw'] / 1e9
cfo1 = data["QuoteSummaryStore"]['cashflowStatementHistory']['cashflowStatements'][0]['totalCashFromOperatingActivities']['raw'] / 1e9

date2 = data["QuoteTimeSeriesStore"]['timeSeries']['annualTotalRevenue'][2]['asOfDate']
rev2 = data["QuoteTimeSeriesStore"]['timeSeries']['annualTotalRevenue'][2]['reportedValue']['raw'] / 1e9
cfo2 = data["QuoteSummaryStore"]['cashflowStatementHistory']['cashflowStatements'][1]['totalCashFromOperatingActivities']['raw'] / 1e9

date3 = data["QuoteTimeSeriesStore"]['timeSeries']['annualTotalRevenue'][1]['asOfDate']
rev3 = data["QuoteTimeSeriesStore"]['timeSeries']['annualTotalRevenue'][1]['reportedValue']['raw'] / 1e9
cfo3 = data["QuoteSummaryStore"]['cashflowStatementHistory']['cashflowStatements'][2]['totalCashFromOperatingActivities']['raw'] / 1e9

date4 = data["QuoteTimeSeriesStore"]['timeSeries']['annualTotalRevenue'][0]['asOfDate']
rev4 = data["QuoteTimeSeriesStore"]['timeSeries']['annualTotalRevenue'][0]['reportedValue']['raw'] / 1e9
cfo4 = data["QuoteSummaryStore"]['cashflowStatementHistory']['cashflowStatements'][3]['totalCashFromOperatingActivities']['raw'] / 1e9
# print(eps, dshrs, eop, rev, ebit, ebitda, ebitda-capex, cfo, round(ebit/rev*100,1), round(ebitda/rev*100,1))
trail_rev = data["QuoteTimeSeriesStore"]['timeSeries']['trailingTotalRevenue'][0]['reportedValue']['raw'] / 1e9
trail_date = data["QuoteTimeSeriesStore"]['timeSeries']['trailingTotalRevenue'][0]['asOfDate']

if len(data["QuoteTimeSeriesStore"]['timeSeries']['annualNormalizedEBITDA']) == 4:
  ebitda1 = data["QuoteTimeSeriesStore"]['timeSeries']['annualNormalizedEBITDA'][-1]['reportedValue']['raw'] / 1e9
  capex1 = data["QuoteSummaryStore"]['cashflowStatementHistory']['cashflowStatements'][0]['capitalExpenditures']['raw'] / -1e9
  ebitda2 = data["QuoteTimeSeriesStore"]['timeSeries']['annualNormalizedEBITDA'][2]['reportedValue']['raw'] / 1e9
  capex2 = data["QuoteSummaryStore"]['cashflowStatementHistory']['cashflowStatements'][1]['capitalExpenditures']['raw'] / -1e9
  ebitda3 = data["QuoteTimeSeriesStore"]['timeSeries']['annualNormalizedEBITDA'][1]['reportedValue']['raw'] / 1e9
  capex3 = data["QuoteSummaryStore"]['cashflowStatementHistory']['cashflowStatements'][2]['capitalExpenditures']['raw'] / -1e9
  ebitda4 = data["QuoteTimeSeriesStore"]['timeSeries']['annualNormalizedEBITDA'][0]['reportedValue']['raw'] / 1e9
  capex4 = data["QuoteSummaryStore"]['cashflowStatementHistory']['cashflowStatements'][3]['capitalExpenditures']['raw'] / -1e9
  trail_ebitda = data["QuoteTimeSeriesStore"]['timeSeries']['trailingNormalizedEBITDA'][0]['reportedValue']['raw'] / 1e9
else:
  ebitda4 = ebitda3 = ebitda2 = ebitda1 = trail_ebitda = None
  capex4 = capex3 = capex2 = capex1 = None
print('TTM', trail_date,trail_rev,trail_ebitda)
print('p1 ',date1,rev1,ebitda1, capex1, cfo1)
print('p2 ',date2,rev2,ebitda2, capex2, cfo2)
print('p3 ',date3,rev3,ebitda3, capex3, cfo3)
print('p4 ',date4,rev4,ebitda4, capex4, cfo4)

TTM 2022-03-31 7.553 None
p1  2021-12-31 5.751 None None 0.746
p2  2020-12-31 8.266 None None 0.888
p3  2019-12-31 6.554 None None 1.828
p4  2018-12-31 8.925 None None 3.062


In [28]:
# url for trailing CF items
url = 'https://finance.yahoo.com/quote/'+ ticker + '/cash-flow?p=' + ticker
page = requests.get(url, headers=headers, timeout=5)
data = json.loads(re.search('root\.App\.main\s*=\s*(.*);', page.text).group(1))["context"]["dispatcher"]["stores"]['QuoteTimeSeriesStore']['timeSeries']

In [ ]:
data.keys() # data['annualCapitalExpenditure'] # list w 4 obj

In [29]:
trail_capex = None
if data['trailingCapitalExpenditure']:
  trail_capex = data['trailingCapitalExpenditure'][0]['reportedValue']['raw'] / -1e9 # list w 1 obj
trail_cfo = data['trailingOperatingCashFlow'][0]['reportedValue']['raw'] / 1e9 # list w 1 obj
trail_date = data['trailingOperatingCashFlow'][0]['asOfDate']
print('TTM',trail_date, trail_capex, trail_cfo)

TTM 2022-03-31 19.705 59.517


#### Net debt

In [35]:
# url for netDebt
url = 'https://finance.yahoo.com/quote/'+ ticker + '/balance-sheet?p=' + ticker
page = requests.get(url, headers=headers, timeout=5)
data = json.loads(re.search('root\.App\.main\s*=\s*(.*);', page.text).group(1))["context"]["dispatcher"]["stores"]["QuoteTimeSeriesStore"]['timeSeries']['annualNetDebt']

In [31]:
data # list w netDebt objs; timeSeries only obj with data, has annual BS items

[]

In [36]:
ndebt1 = ndebt2 = ndebt3 = ndebt4 = None
if len(data) == 4:
  ndebt1 = data[3]['reportedValue']['raw'] / 1e9 if data[3] else None
  ndebt2 = data[2]['reportedValue']['raw'] / 1e9 if data[2] else None
  ndebt3 = data[1]['reportedValue']['raw'] / 1e9 if data[1] else None
  ndebt4 = data[0]['reportedValue']['raw'] / 1e9 if data[0] else None
print(ndebt1, ndebt2, ndebt3, ndebt4)

23.317 26.792 40.186 42.899


In [37]:
# url for industry and sector
url = 'https://finance.yahoo.com/quote/'+ ticker + '/profile?p=' + ticker
page = requests.get(url, headers=headers, timeout=5)
data = json.loads(re.search('root\.App\.main\s*=\s*(.*);', page.text).group(1))["context"]["dispatcher"]["stores"]['QuoteSummaryStore']['assetProfile']
#data.keys()

dict_keys(['zip', 'sector', 'fullTimeEmployees', 'compensationRisk', 'auditRisk', 'longBusinessSummary', 'city', 'phone', 'state', 'shareHolderRightsRisk', 'compensationAsOfEpochDate', 'governanceEpochDate', 'boardRisk', 'country', 'companyOfficers', 'website', 'maxAge', 'overallRisk', 'address1', 'industry'])

In [38]:
sector = data['sector'] # invesment, e.g. WMT = consumer defensive
industry = data['industry'] # E.g. WMT = Discount Stores
print('ind:',industry, 'sec:', sector)

ind: Discount Stores sec: Consumer Defensive


In [39]:
# url for shrs outstanding
url = 'https://finance.yahoo.com/quote/'+ ticker + '/key-statistics?p=' + ticker
page = requests.get(url, headers=headers, timeout=5)
data = json.loads(re.search('root\.App\.main\s*=\s*(.*);', page.text).group(1))["context"]["dispatcher"]["stores"]['QuoteSummaryStore']['defaultKeyStatistics']
data.keys()

dict_keys(['annualHoldingsTurnover', 'enterpriseToRevenue', 'beta3Year', 'profitMargins', 'enterpriseToEbitda', '52WeekChange', 'morningStarRiskRating', 'forwardEps', 'revenueQuarterlyGrowth', 'sharesOutstanding', 'fundInceptionDate', 'annualReportExpenseRatio', 'totalAssets', 'bookValue', 'sharesShort', 'sharesPercentSharesOut', 'fundFamily', 'lastFiscalYearEnd', 'heldPercentInstitutions', 'netIncomeToCommon', 'trailingEps', 'lastDividendValue', 'SandP52WeekChange', 'priceToBook', 'heldPercentInsiders', 'nextFiscalYearEnd', 'yield', 'mostRecentQuarter', 'shortRatio', 'sharesShortPreviousMonthDate', 'floatShares', 'beta', 'enterpriseValue', 'priceHint', 'threeYearAverageReturn', 'lastSplitDate', 'lastSplitFactor', 'legalType', 'lastDividendDate', 'morningStarOverallRating', 'earningsQuarterlyGrowth', 'priceToSalesTrailing12Months', 'dateShortInterest', 'pegRatio', 'ytdReturn', 'forwardPE', 'maxAge', 'lastCapGain', 'shortPercentOfFloat', 'sharesShortPriorMonth', 'impliedSharesOutstandin

In [40]:
shrs_out = data['sharesOutstanding']['raw'] / 1e9
print(shrs_out)

2.741149952


In [ ]:
data['QuoteTimeSeriesStore'].keys()
data['QuoteTimeSeriesStore']['timeSeries'].keys()

In [ ]:
data['QuoteSummaryStore']['financialData'] # price, fins snapshot summary

## Selenium

In [ ]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [ ]:
d = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
ss = input('Stock symbol?')
d.get(f'https://finance.yahoo.com/quote/{ss}/analysis?p={ss}')
main = d.find_element_by_id('Main')
tds = main.find_elements_by_tag_name('td')
if tds:
  data = [e.text for e in tds]
  print(data[0],data[4],data[5],data[8],data[9],data[30],data[33],data[34],data[50],data[53],data[54])
else:
  print('no data')
d.quit()